<a href="https://colab.research.google.com/github/Sulin7254/Python/blob/main/colab/conditional_generation/tev/EvoDiff_conditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evodiff
!pip install torch_geometric
!pip install torch_scatter

In [2]:
from evodiff.pretrained import OA_DM_640M
checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint
model.to("cuda")

Downloading: "https://zenodo.org/record/8045076/files/oaar-640M.tar?download=1" to /root/.cache/torch/hub/checkpoints/oaar-640M.tar
100%|██████████| 7.19G/7.19G [04:20<00:00, 29.7MB/s]


ByteNetLMTime(
  (embedder): ByteNetTime(
    (time_encoding): PositionalEncoding1D()
    (embedder): Embedding(31, 8, padding_idx=28)
    (up_embedder): PositionFeedForward(
      (conv): Conv1d(8, 1280, kernel_size=(1,), stride=(1,))
    )
    (layers): ModuleList(
      (0): ByteNetBlock(
        (conv): MaskedConv1d(1280, 1280, kernel_size=(5,), stride=(1,), padding=(2,))
        (sequence1): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stride=(1,))
          )
          (3): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (4): GELU(approximate='none')
        )
        (sequence2): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stri

In [3]:
# @title 默认标题文本
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/conditional_generation/evodiff_tev/generation_metadata_evodiff_tev.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")



Mounted at /content/drive
Created generation metadata dataframe


In [8]:
!pip install biotite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.6 MB/s eta 0:00:00


In [12]:
!pip show biotite
import biotite
print(biotite.__version__)

Name: biotite
Version: 1.3.0
Summary: A comprehensive library for computational molecular biology
Home-page: https://www.biotite-python.org
Author: The Biotite contributors
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: biotraj, msgpack, networkx, numpy, packaging, requests
Required-by: 
1.3.0


In [14]:
import time
"""
The default generate_scaffold method in evodiff conditional_generation.py places the motif randomly in the scaffold
which is not aligned with out task. I've rewritten the code in order to keep the location of the motif static.
It also can only take pdb ids rather than raw files, but it looks like they just extract the sequence anyway so
we'll just specify the seq.
"""

from evodiff.generate import generate_oaardm
from evodiff.conditional_generation import generate_scaffold

import evodiff
from evodiff.pretrained import OA_DM_640M, OA_DM_38M, CARP_640M, LR_AR_38M, LR_AR_640M
import numpy as np
import argparse
import urllib.request
import torch
import os
import biotite.structure.filter as _bf
import biotite.structure as _bs
_bs.filter_backbone = _bf.filter_peptide_backbone  # 或者根据你需要的 backbone 类型选用 filter_phosphate_backbone



In [16]:
from google.colab import drive
drive.mount('/content/drive')

# 然后创建目录（只需运行一次）
import os
output_dir = '/content/drive/MyDrive/Generative_Models/conditional_generation/evodiff_tev'
os.makedirs(output_dir, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import esm.inverse_folding
from evodiff.utils import Tokenizer, run_omegafold, clean_pdb, run_tmscore #, wrap_dr_bert, read_dr_bert_output
import pathlib
from sequence_models.utils import parse_fasta
from tqdm import tqdm
import pandas as pd
import random
from evodiff.plot import aa_reconstruction_parity_plot, idr_parity_plot

# 设置批次大小和清理GPU缓存
for k in range(0,5):
    batch_size = 10
    torch.cuda.empty_cache()

# 定义元数据
    meta_data = {}
    meta_data["batch_id"] = str(uuid.uuid4())
    meta_data["batch_size"] = str(batch_size)
    meta_data["Timestamp"] = str(datetime.now())
    meta_data["model"] = "evodiff_OA_DM_640M"
    meta_data["task"] = "sequence_generation"
    meta_data["conditions"] = "tev (monomer) scaffolding [motif_seq = GHTTSL#############HLFRR########################################################################################FWKHWIQTKDGQC###############HSASNFTNTNNY################################WGGHKVFMVK, seq_len = 237, start = 27]"
    meta_data["gpu"] = "T4 GPU"
    meta_data['output_file_name'] = None

# 设置生成序列的数量和参数
    num_seqs = batch_size
    motif_seq = "GHTTSL#############HLFRR########################################################################################FWKHWIQTKDGQC###############HSASNFTNTNNY################################WGGHKVFMVK"
    device="cuda"
    seq_len = 237
    start = 27

# 记录开始时间
    start_time = time.time()

# 设置掩码和标记化序列
    mask = tokenizer.mask_id
    motif_tokenized = tokenizer.tokenize((motif_seq,))
    sample = torch.zeros((num_seqs, seq_len))
    sample[:, start:start+len(motif_seq)] = torch.tensor(motif_tokenized)
    nonmask_locations = (sample[0]!= mask).nonzero().flatten()

# 生成序列
    value, loc = (sample == mask).long().nonzero(as_tuple=True)
    loc = np.array(loc)
    np.random.shuffle(loc)
    sample = sample.long().to(device)
    with torch.no_grad():
        for i in loc:
            timestep = torch.tensor([0] * num_seqs)
            timestep = timestep.to(device)
            prediction = model(sample, timestep)
            p = prediction[:, i, :len(tokenizer.all_aas) - 6]
            p = torch.nn.functional.softmax(p, dim=1)
            p_sample = torch.multinomial(p, num_samples=1)
            sample[:, i] = p_sample.squeeze()

# 反标记化序列并记录结束时间
    untokenized = [tokenizer.untokenize(s) for s in sample]
    end_time = time.time()
    total_job_time = end_time - start_time

# 更新元数据
    meta_data["wall_time_batch"] = str(total_job_time) + " Seconds"
    meta_data['wall_time_task'] = str(total_job_time/batch_size) + " Seconds (inferred)"
    print("Generated sequence:", untokenized)

Generated sequence: ['MEPTSPPDSTANDIQYNIDADDDSPHNGHTTSLLSRAKHGYQGYKTHLFRRLRKQIIFVGGILGLLLIILLIVLITIFDNQTKSKSHAYDTVTITPNAQPSQERKLNLGVMLLDTVFKTKDSLVREQCGGSFGATPAIKFWKHWIQTKDGQCFDSYKIDSEGEIFDGHSASNFTNTNNYHKVAYNHISEGTNNEGTFIQLSNLFYGYSNEDWGGHKVFMVKIPCHYGKVPLMFEWGN', 'NTSHEIPHKEEPLLVPFEPSDYPLPRLGHTTSLQDQKVITMSKPNDHLFRRGRGGLIRHFEPDYIIPGNWKHNYYNTAWQIVAWNGPLKFVSIPIWLVTGDKLAPLEDVEMAAQEELAWPSFAQYEGVEIPYFSQEKPEFWKHWIQTKDGQCFTITPTGCELTVWFPHSASNFTNTNNYEDETCSDDSITGALPQGVHRVFFWESDPESTIWGGHKVFMVKSSSEMNQGQILITAAE', 'MQFSMVFIAAATAVYASATPTMRSTTGGHTTSLVKREPNLIPHPSVHLFRRVASIDNRIDAIISDVDHGGDYVAISGCEIDEQLRRHCSAHAIQYNSPGYHKDPRFYDQKNHVLVEGCQPGFPLGPPLGNEGFDDGYADFWKHWIQTKDGQCRKGAREEGNVGVRHIHSASNFTNTNNYAFNPKEVQWNGPHGYSSNAGSIVVGGLHHGNLWGGHKVFMVKNQYNTNSQRRLKEFIH', 'MRIIHILISIATARSELPEETLRSEPAGHTTSLLKVHDSFPDPIFSHLFRRPHISDLVPVNGSAYLLSGLVSNKFSLRKYFVIDTTTMTNKHFLRLFKKSQPKIVLSSNFPPVKSSVVGQKDLGDIEGLVPGWGDATDEFWKHWIQTKDGQCDGDKTTDSLNPSCLGHSASNFTNTNNYVAPCTFYDDPSGYRDVVCSLTKSFSGSLNRAAWGGHKVFMVKYSFNDAFGCQPEPYHK', 'MSWIFLFSQTARAG

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Generative_Models/conditional_generation/evodiff_tev'

In [20]:

  for seq in untokenized:
    meta_data['entity_id'] = str(uuid.uuid4())
    meta_data['generated_sequence'] = seq
    metadata_entry = pd.Series(meta_data)
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
  all_metadata_df.to_csv(meta_data_filepath, index=False)
